##### Copyright 2020 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Cómo escribir un bucle de entrenamiento desde cero

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/guide/keras/writing_a_training_loop_from_scratch">     <img src="https://www.tensorflow.org/images/tf_logo_32px.png">     Ver en TensorFlow.org</a>
</td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/es-419/guide/keras/writing_a_training_loop_from_scratch.ipynb">     <img src="https://www.tensorflow.org/images/colab_logo_32px.png">     Ejecutar en Google Colab</a>
</td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/es-419/guide/keras/writing_a_training_loop_from_scratch.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver código fuente en GitHub</a>
</td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/es-419/guide/keras/writing_a_training_loop_from_scratch.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Descargar bloc de notas</a></td>
</table>

## Preparación

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np

## Introducción

Keras proporciona bucles de entrenamiento y evaluación predeterminados, `fit()` y `evaluate()`. Su uso se explica en la guía [Entrenamiento y evaluación con los métodos incorporados](https://www.tensorflow.org/guide/keras/train_and_evaluate/).

Si desea personalizar el algoritmo de aprendizaje de su modelo sin dejar de aprovechar la comodidad de `fit()` (por ejemplo, para entrenar un GAN utilizando `fit()`), puede subclasificar la clase `Model` e implementar su propio método `train_step()`, que se llama repetidamente durante `fit()`. Esto se explica en la guía <a href="https://www.tensorflow.org/guide/keras/customizing_what_happens_in_fit/" data-md-type="link">Personalización de lo que sucede en `fit()`</a>.

Ahora, si desea un control de muy bajo nivel sobre el entrenamiento y la evaluación, debe escribir sus propios bucles de entrenamiento y evaluación desde cero. De esto trata esta guía.

## Uso de `GradientTape`: un primer ejemplo de extremo a extremo

Llamar a un modelo dentro de un ámbito `GradientTape` permite recuperar los gradientes de los pesos entrenables de la capa con respecto a un valor de pérdida. Al utilizar una instancia del optimizador, puede emplear estos gradientes para actualizar estas variables (que puede recuperar mediante `model.trainable_weights`).

Consideremos un modelo MNIST sencillo:

In [ ]:
inputs = keras.Input(shape=(784,), name="digits")
x1 = layers.Dense(64, activation="relu")(inputs)
x2 = layers.Dense(64, activation="relu")(x1)
outputs = layers.Dense(10, name="predictions")(x2)
model = keras.Model(inputs=inputs, outputs=outputs)

Vamos a entrenarlo utilizando un gradiente mini-lote con un bucle de entrenamiento personalizado.

En primer lugar, necesitaremos un optimizador, una función de pérdida y un conjunto de datos:

In [ ]:
# Instantiate an optimizer.
optimizer = keras.optimizers.SGD(learning_rate=1e-3)
# Instantiate a loss function.
loss_fn = keras.losses.SparseCategoricalCrossentropy(from_logits=True)

# Prepare the training dataset.
batch_size = 64
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_train = np.reshape(x_train, (-1, 784))
x_test = np.reshape(x_test, (-1, 784))

# Reserve 10,000 samples for validation.
x_val = x_train[-10000:]
y_val = y_train[-10000:]
x_train = x_train[:-10000]
y_train = y_train[:-10000]

# Prepare the training dataset.
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(batch_size)

# Prepare the validation dataset.
val_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val))
val_dataset = val_dataset.batch(batch_size)

Este es nuestro bucle de entrenamiento:

- Abrimos un bucle `for` que itera sobre las épocas
- Para cada época, abrimos un bucle `for` que itera sobre el conjunto de datos, en lotes
- Para cada lote, abriremos un entorno `GradientTape()`.
- Dentro de este entorno, llamamos al modelo (paso previo) y calculamos la pérdida
- Fuera del entorno, recuperamos los gradientes de los pesos del modelo con respecto a la pérdida
- Por último, utilizaremos el optimizador para actualizar los pesos del modelo en función de los gradientes

In [ ]:
epochs = 2
for epoch in range(epochs):
    print("\nStart of epoch %d" % (epoch,))

    # Iterate over the batches of the dataset.
    for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):

        # Open a GradientTape to record the operations run
        # during the forward pass, which enables auto-differentiation.
        with tf.GradientTape() as tape:

            # Run the forward pass of the layer.
            # The operations that the layer applies
            # to its inputs are going to be recorded
            # on the GradientTape.
            logits = model(x_batch_train, training=True)  # Logits for this minibatch

            # Compute the loss value for this minibatch.
            loss_value = loss_fn(y_batch_train, logits)

        # Use the gradient tape to automatically retrieve
        # the gradients of the trainable variables with respect to the loss.
        grads = tape.gradient(loss_value, model.trainable_weights)

        # Run one step of gradient descent by updating
        # the value of the variables to minimize the loss.
        optimizer.apply_gradients(zip(grads, model.trainable_weights))

        # Log every 200 batches.
        if step % 200 == 0:
            print(
                "Training loss (for one batch) at step %d: %.4f"
                % (step, float(loss_value))
            )
            print("Seen so far: %s samples" % ((step + 1) * batch_size))

## Administración de métricas de bajo nivel

Incorporemos el monitoreo de métricas a este bucle básico.

Puede reutilizar fácilmente las métricas incorporadas (o las métricas personalizadas que haya escrito) en dichos bucles de entrenamiento escritos desde cero. Este es el flujo:

- Cree la instancia de la métrica al principio del bucle
- Llame a `metric.update_state()` después de cada lote
- Llame a `metric.result()` cuando necesite mostrar el valor actual de la métrica
- Llame a `metric.reset_states()` cuando necesite borrar el estado de la métrica (normalmente cuando termina una época).

Utilicemos este conocimiento para calcular `SparseCategoricalAccuracy` en los datos de validación al final de cada época:

In [ ]:
# Get model
inputs = keras.Input(shape=(784,), name="digits")
x = layers.Dense(64, activation="relu", name="dense_1")(inputs)
x = layers.Dense(64, activation="relu", name="dense_2")(x)
outputs = layers.Dense(10, name="predictions")(x)
model = keras.Model(inputs=inputs, outputs=outputs)

# Instantiate an optimizer to train the model.
optimizer = keras.optimizers.SGD(learning_rate=1e-3)
# Instantiate a loss function.
loss_fn = keras.losses.SparseCategoricalCrossentropy(from_logits=True)

# Prepare the metrics.
train_acc_metric = keras.metrics.SparseCategoricalAccuracy()
val_acc_metric = keras.metrics.SparseCategoricalAccuracy()

Este es nuestro bucle de entrenamiento y evaluación:

In [ ]:
import time

epochs = 2
for epoch in range(epochs):
    print("\nStart of epoch %d" % (epoch,))
    start_time = time.time()

    # Iterate over the batches of the dataset.
    for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):
        with tf.GradientTape() as tape:
            logits = model(x_batch_train, training=True)
            loss_value = loss_fn(y_batch_train, logits)
        grads = tape.gradient(loss_value, model.trainable_weights)
        optimizer.apply_gradients(zip(grads, model.trainable_weights))

        # Update training metric.
        train_acc_metric.update_state(y_batch_train, logits)

        # Log every 200 batches.
        if step % 200 == 0:
            print(
                "Training loss (for one batch) at step %d: %.4f"
                % (step, float(loss_value))
            )
            print("Seen so far: %d samples" % ((step + 1) * batch_size))

    # Display metrics at the end of each epoch.
    train_acc = train_acc_metric.result()
    print("Training acc over epoch: %.4f" % (float(train_acc),))

    # Reset training metrics at the end of each epoch
    train_acc_metric.reset_states()

    # Run a validation loop at the end of each epoch.
    for x_batch_val, y_batch_val in val_dataset:
        val_logits = model(x_batch_val, training=False)
        # Update val metrics
        val_acc_metric.update_state(y_batch_val, val_logits)
    val_acc = val_acc_metric.result()
    val_acc_metric.reset_states()
    print("Validation acc: %.4f" % (float(val_acc),))
    print("Time taken: %.2fs" % (time.time() - start_time))

## Cómo acelerar el paso del entrenamiento con `tf.function`

El tiempo de ejecución predeterminado en TensorFlow 2 es la [ejecución eager](https://www.tensorflow.org/guide/eager). Como tal, nuestro bucle de entrenamiento anterior se ejecuta rápidamente.

Esto es genial para la depuración, pero la compilación de los gráficos tiene una clara ventaja en el rendimiento. Al describir el cálculo como un grafo estático, el marco de trabajo puede implementar optimizaciones de rendimiento globales. Esto es imposible cuando el marco de trabajo se ve obligado a ejecutar una operación tras otra, sin saber lo que viene después.

Puede compilar en un grafo estático con cualquier función que tome tensores como entrada. Solo tiene que agregarle un decorador `@tf.function`, como este:

In [ ]:
@tf.function
def train_step(x, y):
    with tf.GradientTape() as tape:
        logits = model(x, training=True)
        loss_value = loss_fn(y, logits)
    grads = tape.gradient(loss_value, model.trainable_weights)
    optimizer.apply_gradients(zip(grads, model.trainable_weights))
    train_acc_metric.update_state(y, logits)
    return loss_value


Hagamos lo mismo con el paso de la evaluación:

In [ ]:
@tf.function
def test_step(x, y):
    val_logits = model(x, training=False)
    val_acc_metric.update_state(y, val_logits)


Ahora, volvamos a ejecutar nuestro bucle de entrenamiento con este paso de entrenamiento compilado:

In [ ]:
import time

epochs = 2
for epoch in range(epochs):
    print("\nStart of epoch %d" % (epoch,))
    start_time = time.time()

    # Iterate over the batches of the dataset.
    for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):
        loss_value = train_step(x_batch_train, y_batch_train)

        # Log every 200 batches.
        if step % 200 == 0:
            print(
                "Training loss (for one batch) at step %d: %.4f"
                % (step, float(loss_value))
            )
            print("Seen so far: %d samples" % ((step + 1) * batch_size))

    # Display metrics at the end of each epoch.
    train_acc = train_acc_metric.result()
    print("Training acc over epoch: %.4f" % (float(train_acc),))

    # Reset training metrics at the end of each epoch
    train_acc_metric.reset_states()

    # Run a validation loop at the end of each epoch.
    for x_batch_val, y_batch_val in val_dataset:
        test_step(x_batch_val, y_batch_val)

    val_acc = val_acc_metric.result()
    val_acc_metric.reset_states()
    print("Validation acc: %.4f" % (float(val_acc),))
    print("Time taken: %.2fs" % (time.time() - start_time))

Es mucho más rápido, ¿verdad?

## La administración de bajo nivel de las pérdidas registradas por el modelo

Las capas y los modelos realizan un seguimiento recursivo de las pérdidas creadas durante el paso previo por las capas que llaman a `self.add_loss(value)`. La lista correspondiente de valores de pérdidas escalares está disponible mediante la propiedad `model.losses` cuando finaliza el paso previo.

Si desea utilizar estos componentes de pérdida, debe sumarlos y agregarlos a la pérdida principal en su paso en el entrenamiento.

Consideremos esta capa, que crea una pérdida de regularización de la actividad:

In [ ]:
class ActivityRegularizationLayer(layers.Layer):
    def call(self, inputs):
        self.add_loss(1e-2 * tf.reduce_sum(inputs))
        return inputs


Construyamos un modelo muy sencillo que lo utilice:

In [ ]:
inputs = keras.Input(shape=(784,), name="digits")
x = layers.Dense(64, activation="relu")(inputs)
# Insert activity regularization as a layer
x = ActivityRegularizationLayer()(x)
x = layers.Dense(64, activation="relu")(x)
outputs = layers.Dense(10, name="predictions")(x)

model = keras.Model(inputs=inputs, outputs=outputs)

Este es el aspecto que debería tener ahora nuestro paso de entrenamiento:

In [ ]:
@tf.function
def train_step(x, y):
    with tf.GradientTape() as tape:
        logits = model(x, training=True)
        loss_value = loss_fn(y, logits)
        # Add any extra losses created during the forward pass.
        loss_value += sum(model.losses)
    grads = tape.gradient(loss_value, model.trainable_weights)
    optimizer.apply_gradients(zip(grads, model.trainable_weights))
    train_acc_metric.update_state(y, logits)
    return loss_value


## Resumen

Ahora ya conoce todo lo que hay que saber sobre el uso de los bucles de entrenamiento incorporados y sobre cómo escribir los suyos desde cero.

Para concluir, le presentamos un ejemplo sencillo de extremo a extremo que une todo lo que ha aprendido en esta guía: un DCGAN entrenado con dígitos MNIST.

## Ejemplo de principio a fin: un bucle de entrenamiento GAN desde cero

Es posible que conozca las Redes generativas adversariales (GAN). Las GAN pueden generar nuevas imágenes que parecen casi reales, ya que aprenden la distribución latente de un conjunto de datos de imágenes de entrenamiento (el "espacio latente" de las imágenes).

Una GAN se compone de dos partes: un modelo "generador" que asigna puntos en el espacio latente a puntos en el espacio de la imagen, y un modelo "discriminador", un clasificador que puede diferenciar entre imágenes reales (del conjunto de datos de entrenamiento) e imágenes falsas (la salida de la red generadora).

Un bucle de entrenamiento GAN tiene este aspecto:

1. Entrene el discriminador.

- Muestree un lote de puntos aleatorios en el espacio latente.
- Convierta los puntos en imágenes falsas mediante el modelo "generador".
- Obtenga un lote de imágenes reales y combínelas con las imágenes generadas.
- Entrene el modelo "discriminador" para clasificar las imágenes generadas en comparación con las reales.

1. Entrene al generador.

- Muestree puntos aleatorios en el espacio latente.
- Convierta los puntos en imágenes falsas mediante la red "generadora".
- Obtenga un lote de imágenes reales y combínelas con las imágenes generadas.
- Entrene el modelo "generador" para "engañar" al discriminador y clasificar las imágenes falsas como reales.

Para obtener una visión mucho más detallada de cómo funcionan las GAN, consulte [Deep Learning con Python](https://www.manning.com/books/deep-learning-with-python).

Implementemos este bucle de entrenamiento. En primer lugar, cree el discriminador destinado a clasificar los dígitos falsos en comparación con los reales:

In [ ]:
discriminator = keras.Sequential(
    [
        keras.Input(shape=(28, 28, 1)),
        layers.Conv2D(64, (3, 3), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2D(128, (3, 3), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.GlobalMaxPooling2D(),
        layers.Dense(1),
    ],
    name="discriminator",
)
discriminator.summary()

A continuación, crearemos una red generadora que convierta los vectores latentes en salidas con el formato `(28, 28, 1)` (que representan los dígitos MNIST):

In [ ]:
latent_dim = 128

generator = keras.Sequential(
    [
        keras.Input(shape=(latent_dim,)),
        # We want to generate 128 coefficients to reshape into a 7x7x128 map
        layers.Dense(7 * 7 * 128),
        layers.LeakyReLU(alpha=0.2),
        layers.Reshape((7, 7, 128)),
        layers.Conv2DTranspose(128, (4, 4), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2DTranspose(128, (4, 4), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2D(1, (7, 7), padding="same", activation="sigmoid"),
    ],
    name="generator",
)

A continuación, encontrará la parte más importante: el bucle de entrenamiento. Como puede ver, es bastante sencillo. La función de paso de entrenamiento solo ocupa 17 líneas.

In [ ]:
# Instantiate one optimizer for the discriminator and another for the generator.
d_optimizer = keras.optimizers.Adam(learning_rate=0.0003)
g_optimizer = keras.optimizers.Adam(learning_rate=0.0004)

# Instantiate a loss function.
loss_fn = keras.losses.BinaryCrossentropy(from_logits=True)


@tf.function
def train_step(real_images):
    # Sample random points in the latent space
    random_latent_vectors = tf.random.normal(shape=(batch_size, latent_dim))
    # Decode them to fake images
    generated_images = generator(random_latent_vectors)
    # Combine them with real images
    combined_images = tf.concat([generated_images, real_images], axis=0)

    # Assemble labels discriminating real from fake images
    labels = tf.concat(
        [tf.ones((batch_size, 1)), tf.zeros((real_images.shape[0], 1))], axis=0
    )
    # Add random noise to the labels - important trick!
    labels += 0.05 * tf.random.uniform(labels.shape)

    # Train the discriminator
    with tf.GradientTape() as tape:
        predictions = discriminator(combined_images)
        d_loss = loss_fn(labels, predictions)
    grads = tape.gradient(d_loss, discriminator.trainable_weights)
    d_optimizer.apply_gradients(zip(grads, discriminator.trainable_weights))

    # Sample random points in the latent space
    random_latent_vectors = tf.random.normal(shape=(batch_size, latent_dim))
    # Assemble labels that say "all real images"
    misleading_labels = tf.zeros((batch_size, 1))

    # Train the generator (note that we should *not* update the weights
    # of the discriminator)!
    with tf.GradientTape() as tape:
        predictions = discriminator(generator(random_latent_vectors))
        g_loss = loss_fn(misleading_labels, predictions)
    grads = tape.gradient(g_loss, generator.trainable_weights)
    g_optimizer.apply_gradients(zip(grads, generator.trainable_weights))
    return d_loss, g_loss, generated_images


Vamos a entrenar nuestro GAN, llamando varias veces a `train_step` en lotes de imágenes.

Ya que nuestro discriminador y generador son convnets, deseará ejecutar este código en una GPU.

In [ ]:
import os

# Prepare the dataset. We use both the training & test MNIST digits.
batch_size = 64
(x_train, _), (x_test, _) = keras.datasets.mnist.load_data()
all_digits = np.concatenate([x_train, x_test])
all_digits = all_digits.astype("float32") / 255.0
all_digits = np.reshape(all_digits, (-1, 28, 28, 1))
dataset = tf.data.Dataset.from_tensor_slices(all_digits)
dataset = dataset.shuffle(buffer_size=1024).batch(batch_size)

epochs = 1  # In practice you need at least 20 epochs to generate nice digits.
save_dir = "./"

for epoch in range(epochs):
    print("\nStart epoch", epoch)

    for step, real_images in enumerate(dataset):
        # Train the discriminator & generator on one batch of real images.
        d_loss, g_loss, generated_images = train_step(real_images)

        # Logging.
        if step % 200 == 0:
            # Print metrics
            print("discriminator loss at step %d: %.2f" % (step, d_loss))
            print("adversarial loss at step %d: %.2f" % (step, g_loss))

            # Save one generated image
            img = tf.keras.preprocessing.image.array_to_img(
                generated_images[0] * 255.0, scale=False
            )
            img.save(os.path.join(save_dir, "generated_img" + str(step) + ".png"))

        # To limit execution time we stop after 10 steps.
        # Remove the lines below to actually train the model!
        if step > 10:
            break

¡Listo! Obtendrá dígitos MNIST falsos de aspecto agradable después de solo ~30s de entrenamiento en la GPU Colab.